In [1]:
import pandas as pd
import dataframe_image as dfi
import requests
import os
import json
import requests
import warnings
from bs4 import BeautifulSoup
os.makedirs("./Result",exist_ok=True)



# 國內成分證券ETF
https://www.twse.com.tw/zh/products/securities/etf/products/domestic.html

In [4]:


url='https://www.twse.com.tw/rwd/zh/ETF/domestic?response=json&_=1681734086399'

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"
}
req = requests.get(url, headers = headers,verify=False)
ETFdata = req.content.decode('utf8')
ETFdf=pd.DataFrame(json.loads(ETFdata)['data'])
ETFdf=ETFdf.dropna()
ETFdf.columns = ['證券代號', '證券簡稱','']

C:\Users\user\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.twse.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



# ETF   持股 Data

In [5]:
# https://www.moneydj.com/ETF/X/Basic/Basic0007A.xdjhtm?etfid=0050.TW
# https://www.moneydj.com/etf/eb/et305001list.djhtm?order=4&c=47
def get_erfBaseData(etfid,rtfdf):
    etfurl='https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid='+etfid+'.TW'
    etfresp = requests.get(etfurl)
    etfresp.encoding = 'utf-8'    #轉換編碼至UTF-8
    print("utl="+etfurl)
    if etfresp.status_code==200 :
        etfsoup = BeautifulSoup(etfresp.text, 'lxml')
        
        #印出表格中的所有文字內容
        for tr in etfsoup.find('table', id="Repeater1").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            row.append(etfid)
            if len(row)>0:
                rtfdf = rtfdf.append([row],ignore_index=True)
        for tr in etfsoup.find('table', id="Repeater2").find_all('tr'):
            td=tr.find_all('td')
            row = [i.text for i in td]
            if len(row)>0:
                row.append(etfid)
                rtfdf = rtfdf.append([row],ignore_index=True)

        rtfdf=rtfdf.dropna()
       
        return rtfdf

In [6]:
rtfdf = pd.DataFrame()
for  etfid in ETFdf['證券代號'].tolist():
    rtfdf=get_erfBaseData(etfid,rtfdf)
# 【Output】   
rtfdf.columns = ['股票名稱', '持股(千股)','比例','增減','ETF_Id']
rtfdf.to_csv(r'BaseInfo/ETFBaseData.csv', index = False, encoding='utf-8', errors='ignore')
rtfdf.to_csv(r'Result/ETFBaseDataRead.csv', index = False,encoding='big5', errors='ignore')

utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0050.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0051.TW


C:\Users\user\AppData\Local\Temp\ipykernel_5784\1022840050.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_5784\1022840050.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rtfdf = rtfdf.append([row],ignore_index=True)


utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0052.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0053.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0055.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0056.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=0057.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006203.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006204.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=006208.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00690.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00692.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00701.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00713.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xdjhtm?etfid=00730.TW
utl=https://www.moneydj.com/ETF/X/Basic/Basic0007a.xd

In [7]:



StockBaseData=pd.read_csv('BaseInfo/StockBaseData.csv')
ETFBaseData=pd.read_csv('BaseInfo/ETFBaseData.csv')

In [8]:
# 兩表Join
df=pd.merge(ETFBaseData, StockBaseData, how='left', left_on='股票名稱', right_on='公司簡稱')
# 處理格式問題
df['ETF_Id']='00'+df['ETF_Id'].astype(str)

#處理上市公司產業類別對應不到的狀況
df['上市公司產業類別']=df['上市公司產業類別'].fillna('未分類')
print(df.columns)

Index(['股票名稱', '持股(千股)', '比例', '增減', 'ETF_Id', '公司代號', '公司名稱', '公司簡稱', '產業別',
       '實收資本額', 'Type', '上市公司產業類別', '上櫃公司產業類別', '新編碼'],
      dtype='object')


In [9]:
# 取得 etf 對應持股佔比
df['data']=df.agg(lambda x: f"{x['股票名稱']} ( {x['比例']}%)", axis=1)
df.sort_values('比例', ascending=False)
t=df[['ETF_Id','data']]
t[t['ETF_Id']=='00713']

,ETF_Id,data
874,00713,台灣大 ( 9.83%)
875,00713,統一 ( 9.51%)
876,00713,遠傳 ( 9.04%)
877,00713,統一超 ( 6.09%)
878,00713,台新金 ( 3.69%)
879,00713,聯發科 ( 3.39%)
880,00713,華南金 ( 3.35%)
881,00713,中信金 ( 3.19%)
882,00713,玉山金 ( 2.22%)
883,00713,洋基工程 ( 1.94%)


In [10]:
keydata=df.groupby(['ETF_Id','上市公司產業類別'])['data'].apply(','.join).reset_index()
keydata

,ETF_Id,上市公司產業類別,data
0,0050,光電業,大立光 ( 0.57%)
1,0050,其他,中租-KY ( 0.24%)
2,0050,其他電子業,鴻海 ( 4.82%)
3,0050,半導體業,"台積電 ( 57.81%),聯發科 ( 4.7%),日月光投控 ( 1.17%),聯電 ( ..."
4,0050,塑膠工業,"南亞 ( 0.34%),台塑 ( 0.29%),台化 ( 0.21%)"
...,...,...,...
727,00962,資訊服務業,"精誠 ( 1.03%),零壹 ( 0.94%),緯軟 ( 0.17%)"
728,00962,通信網路業,"遠傳 ( 3.11%),美律 ( 1.45%),中磊 ( 1.37%),是方 ( 0.88%..."
729,00962,電子零組件業,"建準 ( 1.43%),群電 ( 0.94%),佳邦 ( 0.45%)"
730,00962,電機機械,"亞德客-KY ( 3.02%),東元 ( 2.1%)"


In [11]:
# 取得產業比例排名 by ETF_Id 上市公司產業類別

# 挑選要用的欄位
colList=['ETF_Id','股票名稱', '上市公司產業類別','持股(千股)','比例']
df_=df[colList]
# groupby
g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()
# 排序 取前三
#g.sort_values('比例', ascending=False).groupby(['ETF_Id']).head(3).sort_values(['ETF_Id', '比例'], ascending=[True, False])
# 排序(這個對未來比較有彈性)
g['row_numbers'] =g.groupby('ETF_Id')['比例'].rank(ascending=False,method='dense')
g[g['ETF_Id']=='00713']

C:\Users\user\AppData\Local\Temp\ipykernel_5784\2400789445.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  g=df_.groupby(['ETF_Id','上市公司產業類別']).sum().reset_index()


,ETF_Id,上市公司產業類別,比例,row_numbers
202,00713,光電業,3.30,9.0
203,00713,其他,4.44,7.0
204,00713,其他電子業,3.11,10.0
205,00713,化學工業,0.97,17.0
206,00713,半導體業,6.21,5.0
207,00713,建材營造,0.50,19.0
208,00713,水泥工業,0.67,18.0
209,00713,汽車工業,1.42,13.0
210,00713,紡織纖維,1.00,16.0
211,00713,航運業,2.03,11.0


In [12]:
#轉成橫的方便看
df1=g[g['row_numbers']==1].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df2=g[g['row_numbers']==2].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df3=g[g['row_numbers']==3].merge(keydata,on=['ETF_Id','上市公司產業類別'])
df_result=df1.merge(df2,on='ETF_Id',suffixes=["","_2"]).merge(df3,on='ETF_Id',suffixes=["","_3"])
df_result = df_result.drop(df_result.filter(regex='row_numbers').columns, axis=1)
df_result

,ETF_Id,上市公司產業類別,比例,data,上市公司產業類別_2,比例_2,data_2,上市公司產業類別_3,比例_3,data_3
0,0050,半導體業,66.61,"台積電 ( 57.81%),聯發科 ( 4.7%),日月光投控 ( 1.17%),聯電 ( ...",金融保險,10.98,"富邦金 ( 1.64%),中信金 ( 1.43%),國泰金 ( 1.4%),兆豐金 ( 1....",電腦及週邊設備業,5.69,"廣達 ( 1.75%),華碩 ( 1.02%),緯創 ( 0.63%),緯穎 ( 0.6%)..."
1,0051,電子零組件業,14.38,"台光電 ( 2.51%),健策 ( 2.1%),嘉澤 ( 1.9%),川湖 ( 1.29%)...",半導體業,11.34,"矽力*-KY ( 1.77%),京元電子 ( 1.65%),創意 ( 1.61%),祥碩 (...",電腦及週邊設備業,10.83,"仁寶 ( 2.04%),技嘉 ( 1.88%),英業達 ( 1.71%),微星 ( 1.67..."
2,0052,半導體業,78.71,"台積電 ( 66.42%),聯發科 ( 5.39%),日月光投控 ( 1.34%),聯電 (...",電腦及週邊設備業,8.50,"廣達 ( 1.96%),華碩 ( 1.12%),緯創 ( 0.73%),緯穎 ( 0.7%)...",其他電子業,6.42,"鴻海 ( 5.64%),可成 ( 0.3%),貿聯-KY ( 0.27%),鴻準 ( 0.21%)"
3,0053,半導體業,65.30,"台積電 ( 52.85%),聯發科 ( 4.94%),日月光投控 ( 1.38%),聯電 (...",電腦及週邊設備業,9.17,"廣達 ( 2.11%),緯穎 ( 0.98%),華碩 ( 0.86%),緯創 ( 0.58%...",電子零組件業,8.09,"台達電 ( 2.88%),嘉澤 ( 0.49%),台光電 ( 0.39%),欣興 ( 0.3..."
4,0056,電腦及週邊設備業,23.64,"華碩 ( 3.14%),和碩 ( 2.47%),廣達 ( 2.45%),仁寶 ( 2.42%...",半導體業,17.83,"瑞昱 ( 3.36%),聯詠 ( 3.33%),聯電 ( 2.83%),聯發科 ( 2.82...",金融保險,14.54,"中信金 ( 3.25%),兆豐金 ( 2.54%),永豐金 ( 2.36%),元大金 ( 2..."
5,0057,半導體業,62.36,"台積電 ( 53.08%),聯發科 ( 4.27%),日月光投控 ( 1.05%),聯電 (...",金融保險,10.47,"富邦金 ( 1.52%),國泰金 ( 1.28%),中信金 ( 1.28%),兆豐金 ( 0...",電腦及週邊設備業,6.34,"廣達 ( 1.52%),華碩 ( 0.85%),緯穎 ( 0.57%),緯創 ( 0.56%..."
6,006203,半導體業,62.45,"台積電 ( 52.99%),聯發科 ( 4.32%),日月光投控 ( 1.07%),聯電 (...",金融保險,10.57,"富邦金 ( 1.48%),國泰金 ( 1.3%),中信金 ( 1.3%),兆豐金 ( 0.9...",電腦及週邊設備業,6.47,"廣達 ( 1.56%),華碩 ( 0.87%),緯穎 ( 0.59%),緯創 ( 0.57%..."
7,006204,半導體業,42.48,"台積電 ( 36.38%),聯發科 ( 3.86%),日月光投控 ( 1.21%),聯電 (...",金融保險,5.09,"富邦金 ( 2.07%),國泰金 ( 1.71%),中信金 ( 1.31%)",其他電子業,3.91,鴻海 ( 3.91%)
8,006208,半導體業,66.67,"台積電 ( 57.89%),聯發科 ( 4.68%),日月光投控 ( 1.17%),聯電 (...",金融保險,10.98,"富邦金 ( 1.64%),中信金 ( 1.43%),國泰金 ( 1.4%),兆豐金 ( 1....",電腦及週邊設備業,5.55,"廣達 ( 1.7%),華碩 ( 0.97%),緯創 ( 0.63%),緯穎 ( 0.6%),..."
9,00690,半導體業,60.44,"台積電 ( 33.38%),聯發科 ( 15.05%),聯電 ( 3.55%),聯詠 ( 2...",電子零組件業,13.21,"台達電 ( 6.41%),國巨 ( 1.76%),台光電 ( 1.36%),嘉澤 ( 1.2...",金融保險,7.56,"富邦金 ( 6.29%),上海商銀 ( 1.27%)"


In [13]:

# export png
df_styled = df_result.style.background_gradient()
df_styled = df_result.style.set_properties(subset=['data','data_2','data_3'], **{'width': '320px'})
dfi.export(df_styled, "Result/mytable.png")

In [14]:
df_styled

,ETF_Id,上市公司產業類別,比例,data,上市公司產業類別_2,比例_2,data_2,上市公司產業類別_3,比例_3,data_3
0,0050,半導體業,66.610000,"台積電 ( 57.81%),聯發科 ( 4.7%),日月光投控 ( 1.17%),聯電 ( 1.13%),聯詠 ( 0.64%),瑞昱 ( 0.6%),世芯-KY ( 0.56%)",金融保險,10.980000,"富邦金 ( 1.64%),中信金 ( 1.43%),國泰金 ( 1.4%),兆豐金 ( 1.02%),玉山金 ( 0.91%),元大金 ( 0.87%),第一金 ( 0.64%),凱基金 ( 0.6%),永豐金 ( 0.6%),合庫金 ( 0.55%),華南金 ( 0.53%),台新金 ( 0.46%),上海商銀 ( 0.33%)",電腦及週邊設備業,5.690000,"廣達 ( 1.75%),華碩 ( 1.02%),緯創 ( 0.63%),緯穎 ( 0.6%),和碩 ( 0.46%),光寶科 ( 0.44%),奇鋐 ( 0.44%),研華 ( 0.35%)"
1,0051,電子零組件業,14.380000,"台光電 ( 2.51%),健策 ( 2.1%),嘉澤 ( 1.9%),川湖 ( 1.29%),健鼎 ( 1.29%),群光 ( 1.19%),華通 ( 1.02%),臻鼎-KY ( 1.01%),金像電 ( 0.98%),信邦 ( 0.73%),南電 ( 0.36%)",半導體業,11.340000,"矽力*-KY ( 1.77%),京元電子 ( 1.65%),創意 ( 1.61%),祥碩 ( 1.15%),力成 ( 1.1%),智原 ( 0.71%),力積電 ( 0.67%),華邦電 ( 0.61%),愛普* ( 0.55%),南亞科 ( 0.45%),采鈺 ( 0.42%),達發 ( 0.38%),威盛 ( 0.27%)",電腦及週邊設備業,10.830000,"仁寶 ( 2.04%),技嘉 ( 1.88%),英業達 ( 1.71%),微星 ( 1.67%),宏碁 ( 1.52%),神達 ( 0.85%),佳世達 ( 0.61%),神基 ( 0.55%)"
2,0052,半導體業,78.710000,"台積電 ( 66.42%),聯發科 ( 5.39%),日月光投控 ( 1.34%),聯電 ( 1.3%),聯詠 ( 0.74%),瑞昱 ( 0.69%),世芯-KY ( 0.64%),矽力*-KY ( 0.34%),京元電子 ( 0.32%),創意 ( 0.31%),祥碩 ( 0.22%),力成 ( 0.21%),智原 ( 0.14%),力積電 ( 0.13%),華邦電 ( 0.12%),愛普* ( 0.11%),南亞科 ( 0.09%),采鈺 ( 0.08%),達發 ( 0.07%),威盛 ( 0.05%)",電腦及週邊設備業,8.500000,"廣達 ( 1.96%),華碩 ( 1.12%),緯創 ( 0.73%),緯穎 ( 0.7%),光寶科 ( 0.51%),奇鋐 ( 0.51%),和碩 ( 0.48%),研華 ( 0.41%),仁寶 ( 0.39%),技嘉 ( 0.36%),英業達 ( 0.33%),微星 ( 0.32%),宏碁 ( 0.29%),神達 ( 0.16%),佳世達 ( 0.12%),神基 ( 0.11%)",其他電子業,6.420000,"鴻海 ( 5.64%),可成 ( 0.3%),貿聯-KY ( 0.27%),鴻準 ( 0.21%)"
3,0053,半導體業,65.300000,"台積電 ( 52.85%),聯發科 ( 4.94%),日月光投控 ( 1.38%),聯電 ( 1.11%),聯詠 ( 0.57%),瑞昱 ( 0.5%),創意 ( 0.48%),聯鈞 ( 0.26%),矽力*-KY ( 0.26%),京元電子 ( 0.22%),祥碩 ( 0.19%),南亞科 ( 0.17%),力成 ( 0.17%),華邦電 ( 0.12%),威盛 ( 0.11%),智原 ( 0.1%),景碩 ( 0.09%),愛普* ( 0.09%),義隆 ( 0.08%),聯陽 ( 0.08%),台勝科 ( 0.08%),矽格 ( 0.08%),致新 ( 0.08%),華泰 ( 0.07%),旺宏 ( 0.07%),創見 ( 0.07%),瑞鼎 ( 0.07%),同欣電 ( 0.07%),力積電 ( 0.07%),京鼎 ( 0.06%),新唐 ( 0.06%),天鈺 ( 0.06%),凌陽 ( 0.05%),超豐 ( 0.05%),立積 ( 0.05%),矽創 ( 0.05%),強茂 ( 0.04%),力智 ( 0.04%),南茂 ( 0.04%),台亞 ( 0.03%),順德 ( 0.03%),菱生 ( 0.03%),偉詮電 ( 0.03%),晶豪科 ( 0.03%),嘉晶 ( 0.03%),敦泰 ( 0.03%),盛群 ( 0.03%),昇陽半導體 ( 0.03%),華東 ( 0.03%),松翰 ( 0.02%),威鋒電子 ( 0.02%),光罩 ( 0.01%),矽統 ( 0.01%),界霖 ( 0.01%)",電腦及週邊設備業,9.170000,"廣達 ( 2.11%),緯穎 ( 0.98%),華碩 ( 0.86%),緯創 ( 0.58%),研華 ( 0.53%),光寶科 ( 0.49%),奇鋐 ( 0.46%),和碩 ( 0.45%),技嘉 ( 0.37%),英業達 ( 0.33%),仁寶 ( 0.32%),微星 ( 0.3%),宏碁 ( 0.24%),神達 ( 0.17%),神基 ( 0.14%),佳世達 ( 0.13%),凌華 ( 0.08%),威強電 ( 0.07%),華擎 ( 0.07%),研揚 ( 0.07%),振樺電 ( 0.06%),友通 ( 0.05%),飛捷 ( 0.05%),尼得科超眾 ( 0.05%),樺漢 ( 0.05%),勤誠 ( 0.05%),精元 ( 0.04%),輔信 ( 0.04%),藍天 ( 0.01%),宏正 ( 0.01%),達方 ( 0.01%)",電子零組件業,8.090000,"台達電 ( 2.88%),嘉澤 ( 0.49%),台光電 ( 0.39%),欣興 ( 0.39%),川湖 ( 0.36%),國巨 ( 0.36%),健策 ( 0.3%),金像電 ( 0.25%),臻鼎-KY ( 0.22%),群光 ( 0.2%),健鼎 ( 0.2%),華通 ( 0.16%),信邦 ( 0.15%),南電 ( 0.15%),新日興 ( 0.12%),群電 ( 0.11%),毅嘉 ( 0.09%),建準 ( 0.09%),華新科 ( 0.09%),精成科 ( 0.08%),信錦 ( 0.07%),晶技 ( 0.07%),康舒 ( 0.07%),日電貿 ( 0.06%),瀚宇博 ( 0.06%),廣宇 ( 0.05%),敬鵬 ( 0.05%),興勤 ( 0.05%),立隆電 ( 0.05%),全漢 ( 0.05%),聯茂 ( 0.05%),台郡 ( 0.05%),志聖 ( 0.04%),健和興 ( 0.04%),禾伸堂 ( 0.04%),嘉聯益 ( 0.03%),嘉基 ( 0.03%),志超 ( 0.03%),凱美 ( 0.02%),鉅祥 ( 0.02%),大毅 ( 0.02%),燿華 ( 0.01%),正崴 ( 0.01%),飛宏 ( 0.01%),致伸 ( 0.01%),鎰勝 ( 0.01%),台虹 ( 0.01%)"
4,0056,電腦及週邊設備業,23.640000,"華碩 ( 3.14%),和碩 ( 2.47%),廣達 ( 2.45%),仁寶 ( 2.42%),微星 ( 2.28%),英業達 ( 2.22%),光寶科 ( 2.21%),技嘉 ( 2.04%),緯創 ( 1.76%),宏碁 ( 1.7%),神基 ( 0.95%)",半導體業,17.830000,"瑞昱 ( 3.36%),聯詠 ( 3.33%),聯電 ( 2.83%),聯發科 ( 2.82%),日月光投控 ( 2.36%),京元電子 ( 1.91%),力成 ( 1.22%)",金融保險,14.540000,"中信金 ( 3.25%),兆豐金 ( 2.54%),永豐金 ( 2.36%),元大金 ( 2.34%),華南金 ( 2.3%),上海商銀 ( 1.75%)"
5,0057,半導體業,62.360000,"台積電 ( 53.08%),聯發科 ( 4.27%),日月光投控 ( 1.05%),聯電 ( 0.95%),聯詠 ( 0.57%),瑞昱 ( 0.54%),世芯-KY ( 0.51%),力旺 ( 0.42%),矽力*-KY ( 0.26%),創意 ( 0.23%),世界 ( 0.21%),環球晶 ( 0.2%),南亞科 ( 0.07%)",金融保險,10.470000,"富邦金 ( 1.52%),國泰金 ( 1.28%),中信金 ( 1.28%),兆豐金 ( 0.9%),玉山金 ( 0.76%),元大金 ( 0.69%),第一金 ( 0.6%),凱基金 ( 0.54%),合庫金 ( 0.51%),永豐金 ( 0.48%),華南金 ( 0.45%),台新金 ( 0.4%),新光金 ( 0.34%),上海商銀 ( 0.3%),彰銀 ( 0.22%),臺企銀 ( 0.2%)",電腦及週邊設備業,6.340000,"廣達 ( 